# Checkpoint 1: Data Cleaning 1
Load the companies and rounds data (provided on the previous page) into two data frames and name them companies and rounds2 respectively.

In [114]:
import pandas as pd
import numpy as np

In [115]:
companies = pd.read_csv('companies.txt', encoding='ISO-8859-1', sep = '\t')
rounds2 = pd.read_csv('rounds2.csv', encoding='ISO-8859-1')

In [116]:
companies.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [117]:
rounds2.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


### company_permalink and permalink columns in rounds2 and companies dataframes are encoded differently and also not case sensitive

In [118]:
rounds2['company_permalink'] = rounds2['company_permalink'].str.encode('utf-8').str.decode('ascii', 'ignore').str.lower().str.strip()
companies['permalink'] = companies['permalink'].str.encode('utf-8').str.decode('ascii', 'ignore').str.lower().str.strip()

In [119]:
companies.shape

(66368, 10)

In [120]:
companies.columns

Index(['permalink', 'name', 'homepage_url', 'category_list', 'status',
       'country_code', 'state_code', 'region', 'city', 'founded_at'],
      dtype='object')

In [121]:
companies.index

RangeIndex(start=0, stop=66368, step=1)

In [122]:
companies.describe()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
count,66368,66367,61310,63220,66368,59410,57821,58338,58340,51147
unique,66368,66102,61191,27296,4,137,311,1092,5111,3978
top,/organization/metreos-corporation,Spire,http://www.askforoffer.com,Software,operating,USA,CA,SF Bay Area,San Francisco,01-01-2012
freq,1,4,5,3995,53034,37601,12900,8804,3526,2730


### As we can see here permalink column has all rows unique, so it can be a unique and primary column

In [123]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [124]:
companies.isnull().sum()

permalink            0
name                 1
homepage_url      5058
category_list     3148
status               0
country_code      6958
state_code        8547
region            8030
city              8028
founded_at       15221
dtype: int64

In [125]:
rounds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114949 entries, 0 to 114948
Data columns (total 6 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(5)
memory usage: 5.3+ MB


In [126]:
rounds2.isnull().sum()

company_permalink              0
funding_round_permalink        0
funding_round_type             0
funding_round_code         83809
funded_at                      0
raised_amount_usd          19990
dtype: int64

### Since approx 20000 rows have nulls in raised_amount_usd column, which turns out to be the fact data
### we need to handle these rows 

In [127]:
rounds2 = rounds2[~(rounds2.isnull().raised_amount_usd)]

In [128]:
rounds2.isnull().sum()

company_permalink              0
funding_round_permalink        0
funding_round_type             0
funding_round_code         66790
funded_at                      0
raised_amount_usd              0
dtype: int64

# Check to see how many rows have all rows null

In [129]:
rounds2.isnull().all(axis=1).sum()

0

In [130]:
rounds2[rounds2.isnull().sum(axis=1) > 2]

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd


### No row is having more than 2 null values in them
# Table 1.1 Answers

## 1. How many unique companies are present in rounds2?
- we use drop_duplicates to remove duplicates from rounds2 using company_permalink column which is primary column for companies dataset
- (Since we have already made this column lowercase so it will exactly match the columns)

In [131]:
len(rounds2.drop_duplicates('company_permalink'))

53915

## 2. How many unique companies are present in the companies file?
- We use drop_duplicates to remove duplicates based on permalink column
- since it is lowercase it will match all unique

In [132]:
len(companies.drop_duplicates('permalink'))

66368

## 3. In the companies data frame, which column can be used as the  unique key for each company? Write the name of the column.

In [133]:
companies.describe()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
count,66368,66367,61310,63220,66368,59410,57821,58338,58340,51147
unique,66368,66102,61191,27296,4,137,311,1092,5111,3978
top,/organization/metreos-corporation,Spire,http://www.askforoffer.com,Software,operating,USA,CA,SF Bay Area,San Francisco,01-01-2012
freq,1,4,5,3995,53034,37601,12900,8804,3526,2730


### As we can see here permalink column has all rows unique, so it can be a unique and primary column

## 4. Are there any companies in the rounds2 file which are not  present in companies ? Answer Y/N.

In [134]:
rounds2[~(rounds2.company_permalink.isin(companies.permalink))]

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd


### Answer is NO. i.e. rounds2 dataset has all the companies present in companies dataset.

## 5. Merge the two data frames so that all  variables (columns)  in the companies frame are added to the rounds2 data frame. Name the merged frame master_frame. How many observations are present in master_frame ?

In [135]:
master_frame = pd.merge(companies, rounds2, how='inner', 
                        left_on='permalink', right_on='company_permalink')
master_frame.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0
4,/organization/01games-technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong,NaN,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,01-07-2014,41250.0


In [136]:
master_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94959 entries, 0 to 94958
Data columns (total 16 columns):
permalink                  94959 non-null object
name                       94958 non-null object
homepage_url               90627 non-null object
category_list              93915 non-null object
status                     94959 non-null object
country_code               89108 non-null object
state_code                 87355 non-null object
region                     87911 non-null object
city                       87914 non-null object
founded_at                 78993 non-null object
company_permalink          94959 non-null object
funding_round_permalink    94959 non-null object
funding_round_type         94959 non-null object
funding_round_code         28169 non-null object
funded_at                  94959 non-null object
raised_amount_usd          94959 non-null float64
dtypes: float64(1), object(15)
memory usage: 12.3+ MB


### 94959 Observations are present in master_frame

# Table 2.1

## Average funding amount of angel, seed, venture and private equity type

In [137]:
group_by_funding = master_frame.groupby(by='funding_round_type')
avg_funding = group_by_funding.mean()

In [138]:
avg_funding.loc[['angel', 'seed', 'venture', 'private_equity']]

,raised_amount_usd
funding_round_type,
angel,9.586945e+05
seed,7.198180e+05
venture,1.174895e+07
private_equity,7.330859e+07


In [139]:
avg_funding.loc[['angel', 'seed', 'venture', 'private_equity']].raised_amount_usd.values

array([  958694.46975309,   719817.99690717, 11748949.12948953,
       73308593.02944215])

In [140]:
avg_funding

,raised_amount_usd
funding_round_type,
angel,9.586945e+05
convertible_note,1.453439e+06
debt_financing,1.704353e+07
equity_crowdfunding,5.383682e+05
grant,4.300576e+06
non_equity_assistance,4.112031e+05
post_ipo_debt,1.687046e+08
post_ipo_equity,8.218249e+07
private_equity,7.330859e+07


## Considering that Spark Funds wants to invest between 5 to 15 million USD per  investment round, which investment type is the most suitable for them?

In [141]:
avg_funding[avg_funding.raised_amount_usd.between(50_00_000, 1_50_00_000)]

,raised_amount_usd
funding_round_type,
venture,1.174895e+07


# Table 3.1

## 1. Top english speaking country

In [142]:
master_frame.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0
4,/organization/01games-technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong,NaN,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,01-07-2014,41250.0


## top nine countries which have received the highest total funding (across ALL sectors for the chosen investment type)

In [143]:
total_funding_agg = master_frame[master_frame.funding_round_type == 'venture'].groupby(by='country_code').sum()

In [144]:
top9 = total_funding_agg.sort_values(by='raised_amount_usd', ascending=False).head(9)

In [145]:
top9

,raised_amount_usd
country_code,
USA,4.225108e+11
CHN,3.983542e+10
GBR,2.024563e+10
IND,1.439186e+10
CAN,9.583332e+09
FRA,7.259537e+09
ISR,6.907515e+09
DEU,6.346960e+09
JPN,3.363677e+09


# Table 4.1

## Checkpoint 4: Sector Analysis

In [146]:
main_sector = pd.read_csv('mapping.csv', encoding='ISO-8859-1')
main_sector.head()

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


In [147]:
main_sector.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 10 columns):
category_list                              687 non-null object
Automotive & Sports                        688 non-null int64
Blanks                                     688 non-null int64
Cleantech / Semiconductors                 688 non-null int64
Entertainment                              688 non-null int64
Health                                     688 non-null int64
Manufacturing                              688 non-null int64
News, Search and Messaging                 688 non-null int64
Others                                     688 non-null int64
Social, Finance, Analytics, Advertising    688 non-null int64
dtypes: int64(9), object(1)
memory usage: 53.8+ KB


In [148]:
main_sector[main_sector.Blanks==1]

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0


In [149]:
main_sector = main_sector.loc[~(main_sector.isnull().category_list)]
main_sector.drop('Blanks', inplace=True,  axis=1)
main_sector.head()

,category_list,Automotive & Sports,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
1,3D,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,1
5,Active Lifestyle,0,0,0,1,0,0,0,0


In [150]:
master_frame.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0
4,/organization/01games-technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong,NaN,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,01-07-2014,41250.0


## Extract primary sector from category_list

In [151]:
master_frame['primary_sector'] = master_frame.category_list.str.split(pat='|').str.get(0)

## We need to unpivot the main_sector (mapping.csv) data so that main_sector can be extracted from 0-1 flags

In [152]:
temp = main_sector.melt(id_vars='category_list', value_name='flag', var_name='main_sector')
main_sector_unpivoted = temp[temp.flag == 1].drop('flag', axis=1)
main_sector_unpivoted.index = range(0, len(main_sector_unpivoted))
main_sector_unpivoted.head()

,category_list,main_sector
0,Adventure Travel,Automotive & Sports
1,Aerospace,Automotive & Sports
2,Auto,Automotive & Sports
3,Automated Kiosk,Automotive & Sports
4,Automotive,Automotive & Sports


## We then need to merge the master_frame with the main_sector_unpivoted dataset

In [153]:
merged_master = master_frame.merge(main_sector_unpivoted, how='inner', left_on='primary_sector', right_on='category_list')
merged_master.head()

,permalink,name,homepage_url,category_list_x,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,primary_sector,category_list_y,main_sector
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,Media,Media,Entertainment
1,/organization/90min,90min,http://www.90min.com,Media|News|Publishing|Soccer|Sports,operating,GBR,H9,London,London,01-01-2011,/organization/90min,/funding-round/21a2cbf6f2fb2a1c2a61e04bf930dfe6,venture,NaN,06-10-2015,15000000.0,Media,Media,Entertainment
2,/organization/90min,90min,http://www.90min.com,Media|News|Publishing|Soccer|Sports,operating,GBR,H9,London,London,01-01-2011,/organization/90min,/funding-round/bd626ed022f5c66574b1afe234f3c90d,venture,NaN,07-05-2013,5800000.0,Media,Media,Entertainment
3,/organization/90min,90min,http://www.90min.com,Media|News|Publishing|Soccer|Sports,operating,GBR,H9,London,London,01-01-2011,/organization/90min,/funding-round/fd4b15e8c97ee2ffc0acccdbe1a98810,venture,NaN,26-03-2014,18000000.0,Media,Media,Entertainment
4,/organization/a-dance-for-me,A Dance for Me,http://www.adanceforme.com/,Media|News|Photo Sharing|Video,operating,USA,MT,Missoula,Missoula,31-07-2011,/organization/a-dance-for-me,/funding-round/9ab9dbd17bf010c79d8415b2c22be6fa,equity_crowdfunding,NaN,26-03-2014,1090000.0,Media,Media,Entertainment


# Checkpoint 5: Sector Analysis 2

## Heavily invested main sectors for each country for venture funcding type and total funding between 5-15 million

In [154]:
venture_funding = merged_master[
    (merged_master.funding_round_type == 'venture') 
    & (merged_master.country_code.isin(['USA', 'GBR', 'IND']))
    & (merged_master.raised_amount_usd.between(5_000_000, 15_000_000))].copy()

### Changing the amount unit in column `raised_amount_usd` to million-dollor

In [155]:
venture_funding.loc[:, 'raised_amount_usd'] = venture_funding.raised_amount_usd.apply(lambda x: x / 1_000_000)

In [156]:
grp = venture_funding.groupby(by=['country_code', 'main_sector'], as_index=False)
grp_agg = grp.raised_amount_usd.agg({'total_investments': 'count', 'total_investment_amount': 'sum'})
master_final = pd.merge(venture_funding, grp_agg, on=['country_code', 'main_sector'])
master_final.head()

,permalink,name,homepage_url,category_list_x,status,country_code,state_code,region,city,founded_at,...,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,primary_sector,category_list_y,main_sector,total_investments,total_investment_amount
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,...,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10.0,Media,Media,Entertainment,33,280.83
1,/organization/dhruva,Dhruva,http://www.dhruva.com/,Games,operating,IND,19,Bangalore,Bangalore,01-01-1997,...,/funding-round/6035248811c9530b11bd442d9239a0b1,venture,NaN,27-11-2006,5.0,Games,Games,Entertainment,33,280.83
2,/organization/games2win,Games2Win,http://www.games2win.com,Games,operating,IND,16,Mumbai,Mumbai,01-01-2005,...,/funding-round/6b024f4906c288c66d1df966e6aeb256,venture,A,29-03-2007,5.0,Games,Games,Entertainment,33,280.83
3,/organization/games2win,Games2Win,http://www.games2win.com,Games,operating,IND,16,Mumbai,Mumbai,01-01-2005,...,/funding-round/b095563fd43d1e4fd16da3f4bcd040af,venture,B,30-03-2011,6.0,Games,Games,Entertainment,33,280.83
4,/organization/pokkt,POKKT,http://www.pokkt.com,Games,operating,IND,16,Mumbai,Mumbai,01-08-2012,...,/funding-round/adb94c131e001a7438a4695d873d8dc1,venture,B,03-11-2015,5.0,Games,Games,Entertainment,33,280.83


In [157]:
D1 = master_final[master_final.country_code == 'USA'].copy()
D2 = master_final[master_final.country_code == 'GBR'].copy()
D3 = master_final[master_final.country_code == 'IND'].copy()

## Table 5.1
### 1. Total number of investments (count)
### 2. Total amount of investment (USD)

In [158]:
grp1 = venture_funding.groupby(by=['country_code'], as_index=False)
grp1.raised_amount_usd.agg({'total_investments': 'count', 'total_investment_amount': 'sum'}).sort_values(by='total_investment_amount', ascending=False)

,country_code,total_investments,total_investment_amount
2,USA,11149,99661.524549
0,GBR,577,5028.704358
1,IND,299,2683.537552


### 3. Top sector (based on count of investments)
### 4. Second-best sector (based on count of investments)
### 5. Third-best sector (based on count of investments)
### 6. Number of investments in the top sector (refer to point 3)
### 7. Number of investments in the second-best sector (refer to point 4)
### 8. Number of investments in the third-best sector (refer to point 5)

In [159]:
grp_agg.sort_values(by=['country_code', 'total_investments'], ascending=False)

,country_code,main_sector,total_investments,total_investment_amount
22,USA,Others,2923,26089.280296
17,USA,Cleantech / Semiconductors,2297,21181.003033
23,USA,"Social, Finance, Analytics, Advertising",1912,16680.695048
21,USA,"News, Search and Messaging",1563,13796.816094
19,USA,Health,908,8200.859357
20,USA,Manufacturing,799,7258.553378
18,USA,Entertainment,580,5000.212982
16,USA,Automotive & Sports,167,1454.104361
14,IND,Others,109,1007.409507
13,IND,"News, Search and Messaging",52,433.834545


### 9. For the top sector count-wise (point 3), which company received the highest investment?

#### we need to find, which company recieved highest investment in `Others` sector for all 3 countries

In [160]:
venture_others = venture_funding[venture_funding.main_sector == 'Others'].copy()
venture_others.sort_values(by='raised_amount_usd', ascending=False, inplace=True)
usa_venture_others = venture_others[venture_others.country_code == 'USA']
gbr_venture_others = venture_others[venture_others.country_code == 'GBR']
ind_venture_others = venture_others[venture_others.country_code == 'IND']

In [161]:
print(usa_venture_others.name.head(1))

19854    1stdibs
Name: name, dtype: object


In [162]:
print(gbr_venture_others.name.head(1))

31841    Azure Solutions
Name: name, dtype: object


In [163]:
print(ind_venture_others.name.head(1))

22338    Pepperfry.com
Name: name, dtype: object


### 10. For the second-best sector count-wise (point 4), which company received the highest investment?

In [164]:
venture_cleantech = venture_funding[venture_funding.main_sector == 'Cleantech / Semiconductors'].copy()
venture_news = venture_funding[venture_funding.main_sector == 'News, Search and Messaging'].copy()

In [165]:
venture_cleantech[venture_cleantech.country_code == 'USA'].name.head(1)

7081    22nd Century Group
Name: name, dtype: object

In [166]:
venture_cleantech[venture_cleantech.country_code == 'GBR'].name.head(1)

7139    Abcodia
Name: name, dtype: object

In [167]:
venture_news[venture_news.country_code == 'IND'].name.head(1)

759    Chillr
Name: name, dtype: object